In [1]:
from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot
import seaborn as sns
import shutil
import importlib

%matplotlib inline

# Display progress logs on stdout
#import logging
#logging.basicConfig(level=logging.DEBUG,
#                   format='%(asctime)s %(levelname)s %(message)s')
#logger = logging.getLogger()
from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import bcolzutils
from bcolzutils import *
importlib.reload(bcolzutils) 


import util
from util import *
importlib.reload(util)

<module 'util' from '/home/tutysara/src/myprojects/mlnd-capstone/util.py'>

In [3]:
# define function to load train, test, and validation datasets
basedir="/media/hdd/datastore/t4sa"
train_idx_path = basedir+ "/b-t4sa_train.txt"
valid_idx_path = basedir+ "/b-t4sa_val.txt"
test_idx_path = basedir+ "/b-t4sa_test.txt"
num_classes = 3

col_names = ["X", "y"]
percent = 0.005
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)

train_data_df = train_data_df[:int(train_data_df.shape[0]*percent)]
valid_data_df = valid_data_df[:int(valid_data_df.shape[0]*percent)]
test_data_df = test_data_df[:int(test_data_df.shape[0]*percent)]

In [4]:
test_data_df.head()

,X,y
0,data/76878/768781748033335296-1.jpg,2
1,data/80174/801747152955326465-1.jpg,0
2,data/76899/768993807845163008-3.jpg,2
3,data/80162/801624637314629634-1.jpg,0
4,data/78083/780835980597194753-1.jpg,2


In [5]:
print(train_data_df.shape)
print(valid_data_df.shape)
print(test_data_df.shape)

print(train_data_df.shape[0] + valid_data_df.shape[0] + test_data_df.shape[0])
sample_df = test_data_df[:5]
sample_df

(1842, 2)
(255, 2)
(255, 2)
2352


,X,y
0,data/76878/768781748033335296-1.jpg,2
1,data/80174/801747152955326465-1.jpg,0
2,data/76899/768993807845163008-3.jpg,2
3,data/80162/801624637314629634-1.jpg,0
4,data/78083/780835980597194753-1.jpg,2


In [6]:
# load TTD
ttd_basedir="/home/tutysara/src/myprojects/senti/dataset"
ttd_data_idx_path = ttd_basedir+ "/twitter_five_agrees.txt"
col_names = ["X", "y"]
ttd_data_df = pd.read_csv(ttd_data_idx_path, sep=" ", header=None, names=col_names)
ttd_data_df.X = ttd_data_df.X.apply(lambda x: ttd_basedir+"/Agg_AMT_Candidates/"+x)
print(ttd_data_df.shape)
ttd_data_df.head()

(882, 2)


,X,y
0,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/28800526...,0
1,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/33547480...,1
2,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/27176682...,1
3,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/10096570...,1
4,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/13247920...,1


In [8]:
bnf_valid_name = 'bottleneck_features_mobilenet_valid'
bnf_test_name = 'bottleneck_features_mobilenet_test' 
bnf_train_name = 'bottleneck_features_mobilenet_train'

valid_name = basedir + '/valid_data'
test_name = basedir + '/test_data'
train_name = basedir +'/train_data' 

In [9]:
# read from disk and check size
valid_data = bcolz.carray(rootdir= valid_name+'_data.bclz', mode='r')
test_data = bcolz.carray(rootdir= test_name + '_data.bclz', mode='r')
train_data = bcolz.carray(rootdir= train_name+ '_data.bclz', mode='r')


valid_labels = bcolz.carray(rootdir= valid_name+'_labels.bclz', mode='r')
test_labels = bcolz.carray(rootdir= test_name + '_labels.bclz', mode='r')
train_labels = bcolz.carray(rootdir= train_name+ '_labels.bclz', mode='r')

print(valid_data.shape)
print(test_data.shape)
print(train_data.shape)

print(valid_labels.shape)
print(test_labels.shape)
print(train_labels.shape)

(51000, 224, 224, 3)
(51000, 224, 224, 3)
(368586, 224, 224, 3)
(51000, 3)
(51000, 3)
(368586, 3)


In [9]:
## make a generator of loaded data
batch_size = 256
valid_data_gen = bcolz_data_generator(valid_data, valid_labels, batch_size=batch_size, progress=True)
test_data_gen = bcolz_data_generator(test_data, test_labels, batch_size=batch_size, progress=True)
train_data_gen = bcolz_data_generator(train_data, train_labels, batch_size=batch_size, progress=True)

In [10]:
## Transform data and save it in bottleneck feacture format

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input as mobilenet_preprocess_input
import time

mobilenet_feature_ext = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
mobilenet_feature_ext._make_predict_function()

s= time.time()
remove_bcolz_dir(bnf_valid_name)
bnf_valid_data,  bnf_valid_labels = bcolz_prediction_writer(gen=valid_data_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_valid_name)
print("Took {:.2f} seconds to calculate bnf_valid_data".format(time.time()-s))

s= time.time()
remove_bcolz_dir(bnf_test_name)
!rm -rf 'bottleneck_features_mobilenet_test*bclz'
bnf_test_data,  bnf_test_labels = bcolz_prediction_writer(gen=test_data_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_test_name)
print("Took {:.2f} seconds to calculate bnf_test_data".format(time.time()-s))

s= time.time()
remove_bcolz_dir(bnf_train_name)
bnf_train_data,  bnf_train_labels = bcolz_prediction_writer(gen=train_data_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        preprocess= mobilenet_preprocess_input,
                        dirname=bnf_train_name)
print("Took {:.2f} seconds to calculate bottleneck_features_mobilenet_train".format(time.time()-s))


Took 2.39 seconds to calculate bnf_valid_data
Took 1.23 seconds to calculate bnf_test_data
bcolz.gen Iteration 0/8 took 1.08s
bcolz.gen Iteration 1/8 took 1.07s
bcolz.gen Iteration 2/8 took 1.09s
bcolz.gen Iteration 3/8 took 1.09s
bcolz.gen Iteration 4/8 took 1.10s
bcolz.gen Iteration 5/8 took 1.08s
bcolz.gen Iteration 6/8 took 1.09s
Took 7.81 seconds to calculate bottleneck_features_mobilenet_train


In [14]:
print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)

print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(255, 7, 7, 1024)
(255, 7, 7, 1024)
(1842, 7, 7, 1024)
(255, 3)
(255, 3)
(1842, 3)


In [13]:
## Read it back from disk and check size
bnf_valid_data = bcolz.carray(rootdir=f'{bnf_valid_name}_data.bclz', mode='r')
bnf_test_data = bcolz.carray(rootdir=f'{bnf_test_name}_data.bclz', mode='r')
bnf_train_data = bcolz.carray(rootdir=f'{bnf_train_name}_data.bclz', mode='r')

bnf_valid_labels = bcolz.carray(rootdir=f'{bnf_valid_name}_labels.bclz', mode='r')
bnf_test_labels = bcolz.carray(rootdir=f'{bnf_test_name}_labels.bclz', mode='r')
bnf_train_labels = bcolz.carray(rootdir=f'{bnf_train_name}_labels.bclz', mode='r')

print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)


print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(255, 7, 7, 1024)
(255, 7, 7, 1024)
(1842, 7, 7, 1024)
(255, 3)
(255, 3)
(1842, 3)


In [16]:
## make a generator of loaded data
batch_size = 256
bnf_train_gen =bcolz_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
bnf_valid_gen =bcolz_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
bnf_test_gen =bcolz_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size) 

In [17]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras import regularizers

alpha = 1
dropout=1e-3

if K.image_data_format() == 'channels_first':
    shape = (int(1024 * alpha), 1, 1)
else:
    shape = (1, 1, int(1024 * alpha))

classes = num_classes

top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 1024)))
top_model.add(Reshape(shape, name='reshape_1'))
top_model.add(Dropout(dropout, name='dropout'))
top_model.add(Conv2D(classes, (1, 1),
           padding='same', name='conv_preds'))
top_model.add(Activation('softmax', name='act_softmax'))
top_model.add(Reshape((classes,), name='reshape_2'))

top_model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1024)        0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 3)           3075      
_________________________________________________________________
act_softmax (Activation)     (None, 1, 1, 3)           0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 3)                 0         
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
top_model.fit_generator(bnf_train_gen,
          steps_per_epoch= (1 + (bnf_train_data.shape[0]// batch_size)),
          epochs=15,
          validation_data=bnf_valid_gen,
          validation_steps= (1 + (bnf_valid_data.shape[0] // batch_size)),
          callbacks=[early_stopping, checkpointer])

Epoch 1/15
8/8 [==============================] - 1s 141ms/step - loss: 1.4170 - acc: 0.3202 - val_loss: 1.3607 - val_acc: 0.2863
Epoch 2/15
8/8 [==============================] - 1s 78ms/step - loss: 1.3001 - acc: 0.3725 - val_loss: 1.3199 - val_acc: 0.3137
Epoch 3/15
8/8 [==============================] - 1s 82ms/step - loss: 1.1965 - acc: 0.4028 - val_loss: 1.2490 - val_acc: 0.3725
Epoch 4/15
8/8 [==============================] - 1s 88ms/step - loss: 1.1523 - acc: 0.4331 - val_loss: 1.2465 - val_acc: 0.3608
Epoch 5/15
8/8 [==============================] - 1s 86ms/step - loss: 1.1039 - acc: 0.4415 - val_loss: 1.1951 - val_acc: 0.3725
Epoch 6/15
8/8 [==============================] - 1s 82ms/step - loss: 1.0688 - acc: 0.4583 - val_loss: 1.1883 - val_acc: 0.3686
Epoch 7/15
8/8 [==============================] - 1s 87ms/step - loss: 1.0343 - acc: 0.4924 - val_loss: 1.1677 - val_acc: 0.3882
Epoch 8/15
8/8 [==============================] - 1s 87ms/step - loss: 1.0111 - acc: 0.5069 - va

In [24]:
y_true, y_pred = prediction_from_gen(gen=bnf_test_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=top_model,
                                     dirname="bottleneck_features_mobilenet_test")
                                    


In [25]:
print(y_true.shape)
print(y_pred.shape)

(255, 3)
(255, 3)


In [26]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 45.0980%


In [27]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet_preprocess_input(paths_to_tensor(ttd_X1))
ttd_X = mobilenet_feature_ext.predict(ttd_X2)

In [28]:
ttd_y_pred = top_model.predict(ttd_X)
ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [29]:
print(ttd_X.shape)
print(ttd_y_pred.shape)
print(ttd_y_true.shape) 

(882, 7, 7, 1024)
(882, 3)
(882,)


In [30]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 58.6168%


In [31]:
# fine tune on full model
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

mobilenet_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# CREATE AN "REAL" MODEL FROM Mobilenet
# BY COPYING ALL THE LAYERS OF Mobilenet
model = Sequential()
for l in mobilenet_model.layers:
    model.add(l)


# CONCATENATE THE TWO MODELS
model.add(top_model)

# LOCK THE TOP CONV LAYERS
for layer in model.layers:
    layer.trainable = False

# COMPILE THE MODEL
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [32]:
# load train, test, and validation datasets
batch_size = 256
valid_data_gen = bcolz_data_generator(valid_data, valid_labels, batch_size=batch_size, progress=False)
test_data_gen = bcolz_data_generator(test_data, test_labels, batch_size=batch_size, progress=False)
train_data_gen = bcolz_data_generator(train_data, train_labels, batch_size=batch_size, progress=False)

from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fullmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit_generator(train_data_gen,
          steps_per_epoch= (1+ train_data_df.shape[0]// batch_size),
          epochs=2,
          validation_data=valid_data_gen,
          validation_steps= (1+ valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

Epoch 1/2
8/8 [==============================] - 9s 1s/step - loss: 0.9668 - acc: 0.5408 - val_loss: 1.2030 - val_acc: 0.4039
Epoch 2/2
8/8 [==============================] - 5s 671ms/step - loss: 0.9680 - acc: 0.5439 - val_loss: 1.1975 - val_acc: 0.4118


In [33]:
y_true, y_pred = prediction_from_gen(gen=test_data_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=model,
                                     dirname="mobilenet_test")
                                    

In [34]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 42.3529%


In [37]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet_preprocess_input(paths_to_tensor(ttd_X1))
ttd_y_pred = model.predict(ttd_X2)
    
ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [38]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 34.0136%
